
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/main/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

Получить свой бредовый текст, прямо как в канале [Машинное отупение](https://t.me/MachineScorning)!

In [1]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "Нейросвояк"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом", "Нейроюморески", "Нейросвояк"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kishverses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings",
    "Нейроюморески": "neurojumoreski",
    "Нейросвояк": "neurosvoyak"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

neurosvoyak.tar.gz  100%[===================>] 443.70M   117MB/s    in 4.0s    
Нейросвояк: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [2]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [3]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [4]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

Загружены примеры по умолчанию


In [5]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов =   3#@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [6]:
!nvidia-smi

Sun Mar 14 12:21:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [7]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 133kB 20.1MB/s 
     |████████████████████████████████| 573kB 37.9MB/s 
     |████████████████████████████████| 5.6MB 51.3MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
     |████████████████████████████████| 133kB 60.5MB/s 
     |████████████████████████████████| 1.2MB 53.5MB/s 
     |████████████████████████████████| 7.3MB 51.6MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [8]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [9]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [15]:
#@markdown ### Начать генерацию!

Бредовость = 25 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = True   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  if Модель == "Нейросвояк":
    rs = re.sub(r"\s([0-9]0?)\.", r"<br><br>\1\\.", rs)
    rs = rs.replace("1. ", "1\. ")
    rs = re.sub(r"\s((?:Ответ: )|(?:Источник[а-яё()]: ))", r"<br>\1", rs)
    sq = re.search(r"Автор[а-яё()]?: ", rs)
    if sq:
      rs = rs[:sq.start() - 1]
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

Ключ случайного значения: **42**

<b>ТЕМА КОНЬЯ</b>

1\. В стихотворении Михаила Светлова ОН назван "Кузнец в тигровой шкуре".<br>Ответ: Кий.<br><br>2\. По словам героя песни, ЭТОТ КОМБИНАТОР, будучи в командировке, "написал стихотворение, в котором описал свой быт в Сибири, трудовые подвиги, которые он совершил".<br>Ответ: Сергей Есенин.<br><br>3\. ЭТОМУ ЧЕЛОВЕКУ, жившему в начале XX века в Петербурге, в начале 1920-х годов пришла на ум фраза из фильма "Москва слезам не верит".<br>Ответ: Николай Гумилёв.<br><br>4\. В ответе — два существительных. Согласно русской пословице, К НЕМУ приходит беда, если к нему никто не пришел.<br><br>5\. В стихотворении Владимира Маяковского ОН "приходит в контору с пустыми руками".... <br>Ответ: Кузнец (песня "Принеси хлеба и зрелищ").<br>

<b>ТЕМА КОНЬЯ</b>

1\. ОН был сыном крестьянина и получил свое имя по имени его матери-одиночки.<br>Ответ: КОНЕЦ.<br><br>2\. ЭТУ КОНФЕТУ в конце 80-х годов XIX века изобрела советская теннисистка Ольга Неумывакина в Ташкенте.<br>Ответ: БРАЗИЛИЯ [зачет: БАРЗИЛИЯ]<br><br>3\. Именно в этот период времени в России начала формироваться концепция о том, что в семье должен быть минимум три обезьяны.<br>Ответ: К концу 19 века (требовалась четкая привязка к семье — как к обезьяне).<br><br>4\. ЭТО СЛОВО первоначально обозначало "конюшне", а в 17 в. его часто называют просто "кружком".<br>Ответ: КОРОБКУ.<br><br>5\. ЭТОТ КАНАДЕЦ династии Мин в 1686 году подарил свою жену Екатерину Алексеевну, сестру Александра I, а в 1812 г. был назначен членом верховного совета русской армией.... <br>Ответ: КОНСТАНТИН I.<br>

...

1\. В ответе два слова. Ответ на этот вопрос должен состоять из двух слов. В одном из вопросов ОН был не один, а со своей собакой.<br>Ответ: Руслан из сериала "Доктор Хаус".<br><br>2\. ЭТОТ русский КОМПОЗИТОР был убит в результате покушения на своего кузена князя Курбского, в котором принимал самое активное участие.<br>Ответ: [Василий Михайлович] Татищев.<br><br>3\. По мнению одного из персонажей, ЭТОТ человек не смог бы жить, не будучи съеден собственным мясом.<br>Ответ: [Иван] Васильевич] Столыпин, который был отравлен ядом.<br><br>4\. Именно ОН, согласно русской пословице, не птица.<br>Ответ: Иван Федорович Крузенштерн ("Петр Иванович").<br><br>5\. Именно ТАК по-французски звучит слово mort. В русском языке его обычно называют "малыш" или "девочка", а в английском языке оно обозначает "женщина".... <br>Ответ: "Поркюльоне". Источник(и):<br><br>1\. http://en.wikipedia.org/wiki/Ruslan_(super-fascist;<br><br>2\. http://ru.wikipedia.org/wiki/Доктор_Хаус:_Русское_искусство;<br><br>3\. http://en.wikipedia.org/wiki/Porcullon_(mallet);<br><br>4\. http://en.wikipedia.org/wiki/Porcullon;<br><br>5\. Мультфильм "Русалочка".<br>

<b>ВОПРОСЫ ОТ ЗАКЛАДОЧНИКА РУСЛАНА</b>

1) Что в России так любят говорить, что в ней нет ни грамма романтики<br>Ответ: "Романовская" 2) Что в России называют словом "русское слово" и "великая литература", почему оно не встречается во всех школьных учебниках по истории? 3) Как называется книга о Роланде? 4) Кто был прототипом Роланда? 5) Как называется фильм, рассказывающий о Роланде как о Наполеоне<br>Ответ: Роллан-д'Ивуар (пьеса называется "Романс Наполеона"...) 6) Что означает слово "русская литература" в "Евгенье онеге""... 
Комментарий: Речь шла о романе Роланда "Русалка" [зачет: по упоминанию романа "Русалки"] (Роман назывался "Роман о Роланде"). Источник(и):<br><br>1\. "Советский энциклопедический словарь" под редакцией В.А.Кузищина, Н.Н.Суперанской, В.Н.Куприяновой, А.Н.Ботвинником и Н.И.Жеваго.;<br><br>2\. Анекдот. — Т.<br><br>2\.;<br><br>3\. БЭС, 1988.;<br><br>4\. Л.Воробьев "Этюды о Роланде" и другие сочинения русских литераторов. — М.: Политиздат, 1967;<br><br>5\. http://lib.rus.ec/g/science/science_1985_read_3.shtml<br>

<b>ВОПРОСЫ ОТ ЗАКЛАДОЧНИКА РУСЛАНА</b>

1\. В этом рассказе Николая Васильевича Бунина рассказывается о жизни и подвигах русских первопроходцев через Северный полюс ионосферный шар в течение нескольких столетий после открытия Америки.<br><br>2\. Этот русский поэт был участником экспедиции в Антарктиду.<br>Ответ: Александр Грин.<br><br>3\. В этом произведении Николая Васильевича Бунина рассказывается о жизни и подвигах двух русских первопроходцев на Северный полюс, о жизни и творчестве Николая Гумилева.<br>Ответ: "Похождения бравого солдата Швейка в солдаты" {незачет: "Похождения бравого солдата Швейка в солдаты"}.<br><br>4\. Именно этот вопрос задают себе люди, ответившим именно так.... <br>Ответ: Что им ответить?<br>Ответ: "Что они хотят? Что им нужно?" {зачет: по цитате или по упоминанию вопроса}.<br><br>5\. Именно этот вопрос задается тем же вопросом у Николая Васильевича в романе "Двенадцать стульев".... <br>Ответ: "А как называется город на Северном полюсе?".<br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [ ]:
%%writefile input.txt


Overwriting input.txt


In [16]:
#@markdown ### Генерируем новую выдачу

Повторов = 3  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Многовато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 25 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  if Модель == "Нейросвояк":
    rs = re.sub(r"\s([0-9]0?)\.", r"<br><br>\1\\.", rs)
    rs = rs.replace("1. ", "1\. ")
    rs = re.sub(r"\s((?:Ответ: )|(?:Источник[а-яё()]: ))", r"<br>\1", rs)
    sq = re.search(r"Автор[а-яё()]?: ", rs)
    if sq:
      rs = rs[:sq.start() - 1]
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

Ключ случайного значения: **6724**

<b>ТЕМА КОНЬЯ</b>

(В каждом ответе — по два слова.)<br><br>1\. ЭТОТ КЛУБ, единственный из российских, в который входят два футбольных клуба, носит имя своего основателя — Владимира Ивановича Коровина.<br>Ответ: ККМ.<br><br>2\. Именно ТАК в Древней Руси назывался воинский чин.<br>Ответ: Князь.<br><br>3\. ЭТОТ ЧЕЛОВЕК, один из создателей теории эволюции, первым в России выступил на научной конференции по проблеме происхождения видов и предложил методы селекции и воспроизводства.<br>Ответ: К.Н.Румянцев.<br><br>4\. ЭТОТ КЛУБ — единственная команда России, которая выступает на чемпионатах Европы и мира, а еще один клуб в КХЛ. А вот один из его игроков в прошлом году выступал на чемпионатах России.<br>Ответ: "СиБ" (Борис Березовский, "Сибиряков").<br><br>5\. ОН является автором романа "Собор парижской богоматери".... <br>Ответ: Мишель де Бовуар. Источник(и):<br><br>3\. "Советский спорт". 1992. — Т. 1, с. 475.;<br><br>4\. В. И. Ленин. Энциклопедический Словарь. — С. 329, 328.;<br><br>5\. "Комсомольская правда" от 26.09.99 г.<br>

<b>ТЕМА КОНЬЯ</b>

1\. В ЭТОМ СЕРИАЛЕ фигурирует "Кон-Тики" и, конечно, сам король.<br>Ответ: [Кукольный] мюзикл [мультсериал] "Фантастическое путешествие".<br><br>2\. ЭТОГО ЧЕЛОВЕКА называли и в честь "королевы-вороны", и в честь "короля-рыбу".<br>Ответ: Кондор [Финнеса] {зачет: Тор (королевич Кондор)} (фильм "Король-рыба").<br><br>3\. ЭТОТ ПОЛУЗАЩИТНИК, в честь которого названа "королева-рыба" и который является одним из главных героев мультфильма, получил в 2006 году Нобелевскую премию по литературе.<br>Ответ: [Кондор] Дюрвиль.<br><br>4\. ЭТОМУ ЧЕЛОВЕКУ приписывают создание "Королевы меча и магии".<br>Ответ: [Кондор] [Кэрол].<br><br>5\. В честь ЭТОГО ЧЕЛОВЕКА названы и "Конь в мехах", и "Конь в доспехах" в одноименной повести Александра Беляева.... <br>Ответ: [Конкорд] [Роберт] де Филип. Источник(и): 1-5. БЭКМ, 2000.;<br><br>2\. http://lib.ru/INPROJECTION/CHUBARY_DEPTION;<br><br>3\. http://www.flibusta.net/b/294498/read;<br><br>4\. http://lib.ru/INPROJECTION/chuka/book.txt;<br><br>5\. http://ru.wikipedia.org/wiki/Филип<br>

...

1\. Именно этот советский скульптор является автором картины, на которую были вынесены слова: "Я не знаю, кто эта девушка, но она мне очень нравится"<br>Ответ: [Элизабет] Руссо<br><br>2\. В стихотворении, написанном именно под влиянием ЭТОЙ ПРОЦЕДУРЫ, говорится, что "наше время, время великих перемен".<br>Ответ: Перемены<br><br>3\. В ответе надо указать имя человека.<br>Ответ: [Лев Давидович] Троцкий<br><br>4\. Именно он написал картину с упоминанием темы.<br>Ответ: [Владимир Ильич) Чижевский<br><br>5\. На самом деле это были слова "Я не знаю", но, по-видимому, они были произнесены вслух.... <br>Ответ: Владимир Ленин<br>

<b>ВОПРОСЫ ОТ ЗАКЛАДОЧНИКА РУСЛАНА</b>

(Тема посвящена людям с очень высоким IQ.) В начале своей книги О НЕМ Владимир Набоков сказал: "Он не из тех, кому легко с собой....Он просто человек с очень крепким здоровьем, который много знает, много читает, но никогда никому не говорит об этом".... <br>Ответ: Михаил БУХАРИН (зачет: Михаил БУХАРИН-младший и Александр БУХАРИН-младший соответственно; Набоков назвал Михаила БАРИНА человеком с крепким сердцем, который много знает, много пишет, но никогда никому не говорит об этом). Источник(и):<br><br>1\. https://books.google.ru/books?id=kAQJgAwAQ&pg=PAA#v=onepage&q&f=false<br>

<b>ВОПРОСЫ ОТ ЗАКЛАДОЧНИКА РУСЛАНА</b>

1\. Этот вопрос задается на странице Википедия, а не в разделе "Ссылки", как в ответе. Ответ "О чем речь?".<br><br>2\. В этой песне группы "Наутилус Помпилиус" упоминается, в частности, вопрос: а где же находится эта улица.<br>Ответ: На Васильевском Острове. Зачет: По аналогии с "Анастасиевской".<br><br>3\. Этот советский киноактер был режиссером-постановщиком картины о советском школьнике В.И. Чапаеве и сыграл его в фильмах "Победа", "Демон" и других.<br>Ответ: Александр Татарский ("Поручик Казанский").<br><br>4\. В фильме Александра Казанского в фильме "Остров Сахалин" рассказывается, как на берегу Тихого океана русский офицер Иван Федорович находит в развалинах заброшенную мельницу.<br>Ответ: "Остров погибших моряков" (фильмы "Сотворение мира" (1953), "Человек и закон" (1957), "Человек в футуре" (1984), "Человек в черном ботинке" и многие другие).<br><br>5\. В этой повести писателя Михаила Жванецкого рассказывается, как на самом деле звали отца Александра Сергеевича.... <br>Ответ: "О чем говорят мужчины".
<br>

На этом всё. Получайте удовольствие (надеюсь)!